# Diving into Pinecone

Last Update: Jan 10, 2024

In [1]:
# loading the API keys in environment variables
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [6]:
pip install -q pinecone-client<3

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pinecone

# initializing pinecone
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),
    environment=os.environ.get('PINECONE_ENV')
)

In [8]:
pinecone.info.version()

VersionResponse(server='2.0.11', client='2.2.4')

### Pinecone Indexes

In [9]:
pinecone.list_indexes()

['langchain-pinecone']

#### Creating an index

In [10]:
# creating an index
index_name = 'langchain-pinecone'
if index_name not in pinecone.list_indexes():
    print(f'Creating index {index_name} ....')
    pinecone.create_index(index_name, dimension=1536, metric='cosine', pods=1, pod_type='p1.x2')
    print('Done')
else:
    print(f'Index {index_name} already exists!')

Index langchain-pinecone already exists!


In [11]:
pinecone.describe_index(index_name)

IndexDescription(name='langchain-pinecone', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='p1.x1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

#### Deleting an index

In [13]:
# deleting an index
index_name = input('Enter Pinecone index to delete: ')
if index_name in pinecone.list_indexes():
    print(f'Deleting index {index_name} ... ')
    pinecone.delete_index(index_name)
    print('Done')
else:
    print(f'Index {index_name} does not exist!')

Enter Pinecone index to delete: react
Deleting index react ... 
Done


#### Getting index statistics

In [12]:
# getting statistics
index_name = 'langchain-pinecone'
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5}

#### Inserting into an index

In [13]:
# inserting some random vectors into a Pinecone index
import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]

# vectors id
ids = list('abcde')

In [20]:
index_name = 'langchain-pinecone'
index = pinecone.Index(index_name)
index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

#### Updating a vector

In [14]:
# updating a vector
index.upsert(vectors=[('c', [0.3] * 1536)])

{'upserted_count': 1}

#### Fetching a vector

In [15]:
# fetching a vector
index = pinecone.Index('langchain-pinecone')
index.fetch(ids=['c', 'd'])

{'namespace': '',
 'vectors': {'c': {'id': 'c',
                   'values': [0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
       

#### Deleting vectors

In [16]:
# deleting vectors
index.delete(ids=['b', 'c'])

{}

In [17]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3}

In [18]:
index.fetch(ids=['b'])

{'namespace': '', 'vectors': {}}

In [25]:
index.delete(delete_all=True)

{}

#### Querying

In [19]:
# querying 
queries = [[random.random() for _ in range(1536)] for v in range(2)]

In [20]:
index.query(
    queries=queries,
    top_k=3,
    include_values=False
)

{'results': [{'matches': [{'id': 'd', 'score': 0.741436601, 'values': []},
                          {'id': 'a', 'score': 0.740033805, 'values': []},
                          {'id': 'e', 'score': 0.735506594, 'values': []}],
              'namespace': ''},
             {'matches': [{'id': 'e', 'score': 0.747635543, 'values': []},
                          {'id': 'd', 'score': 0.740624964, 'values': []},
                          {'id': 'a', 'score': 0.737371266, 'values': []}],
              'namespace': ''}]}